In [ ]:
import usb.core
import usb.util

def list_usb_devices():
    # Find all connected USB devices
    devices = usb.core.find(find_all=True)

    # Iterate over each device and print information
    for device in devices:
        print(f"Device: {device}")
        print(f"  - idVendor: {hex(device.idVendor)}")
        print(f"  - idProduct: {hex(device.idProduct)}")
        print(f"  - Manufacturer: {usb.util.get_string(device, device.iManufacturer)}")
        print(f"  - Product: {usb.util.get_string(device, device.iProduct)}")
        print(f"  - Serial Number: {usb.util.get_string(device, device.iSerialNumber)}")
        print()

if __name__ == "__main__":
    list_usb_devices()


In [ ]:
def find_printer(vendor_id, product_id):
    # Find the USB device
    device = usb.core.find(idVendor=vendor_id, idProduct=product_id)

    if device is None:
        raise ValueError("Printer not found")

    # Detach it from kernel driver if necessary
    if device.is_kernel_driver_active(0):
        device.detach_kernel_driver(0)

    # Set the active configuration
    device.set_configuration()
    return device

def print_sequence(device, number):
    print('tarek')
    # Initialize printer (ESC @)
    data = b'\x1b\x40'
    data += b'\x1b\x37\x06\x15\x10'  # ESC 7 - Set heating parameters for dark, high-quality prints    

    # Print a title in double size
    data += b'\x1b\x21\x30'  # ESC ! n -- Select print mode for double height and width
    data += 'Your comany name\n'.encode('utf-8')

    # Reset to normal size text
    data += b'\x1b\x21\x00'  # ESC ! n -- Cancel character size
    data += 'Registration\n'.encode('utf-8')

    # Reset to normal size text
    data += b'\x1b\x21\x10'  # ESC ! n -- Select print mode for double height and width
    data += 'INFO : \n'.encode('utf-8')

    # Draw a horizontal line or border
    data += b'\x1b\x21\x00'  # ESC ! n -- Select print mode for double height and width
    data += b'--------------------------------\n'

    # Token number in large size
    data += b'\x1b\x21\x30'  # ESC ! n -- Select print mode for double height and width
    data += 'TOKEN NO:{}\n'.format(number).encode('utf-8')

    # Reset to normal size text
    data += b'\x1b\x21\x10'  # ESC ! n -- Cancel character size

    # Cut the paper with a partial cut (GS V m)
    data += b'\x1d\x56\x42\x00'  # GS V m -- Cut the paper with a partial cut

    # Try to disable the beeper
    data += b'\x1B\x42\x00\x00'  # ESC B 0 0 -- Attempt to disable the beeper

    # Write data to the printer
    endpoint_address = 0x03  # Endpoint address for Bulk OUT
    device.write(endpoint_address, data)

def main():
    vendor_id = 0x0483 ## set this as. per your device
    product_id = 0x5720 ## set this as. per your device

    # Find the printer
    printer = find_printer(vendor_id, product_id)

    # Print the sequence
    print_sequence(printer, 123)  # Example token number

    print("Sequence printed to printer")

if __name__ == "__main__":
    main()